In [1]:
using PorousMaterials, LightGraphs

include("src/moiety.jl")
fragment = moiety("p-phenylene")

xtal = Crystal("IRMOF-1.cif")
strip_numbers_from_atom_labels!(xtal)
infer_bonds!(xtal, false)
xtal.bonds

subgraph = fragment.bonds
graph = xtal.bonds

{424, 360} undirected simple Int64 graph

In [2]:
# "compatibility matrix"
# M₀[α, β] = 1 if any only if:
#     deg(β ∈ graph) ≥ deg(α ∈ subgraph)
#      and
#     species(α ∈ subgraph) == species(β ∈ graph)
function compatibility_matrix(subgraph::SimpleGraph, subgraph_species::Array{Symbol, 1},
                              graph::SimpleGraph,    graph_species::Array{Symbol, 1})::Array{Bool, 2}
    @debug "Finding M₀..."
    # allocate M. rows correspond to subgraph nodes, columns to graph nodes.
    M₀ = zeros(Bool, nv(subgraph), nv(graph))
    for α ∈ 1:nv(subgraph) # Loop over rows (subgraph nodes)
        for β ∈ 1:nv(graph) # Loop over columns (graph nodes)
            # Record Bool for each (i,j): true if atom species match and graph node degree is sufficient.
            if (degree(graph, β) ≥ degree(subgraph, α)) && (subgraph_species[α] == graph_species[β])
                M₀[α, β] = true # cannot rule out correspondence, so, true, for all we know at this point, these could correspond.
            end
        end
    end
    return M₀
end

M₀ = compatibility_matrix(fragment.bonds, fragment.atoms.species, xtal.bonds, xtal.atoms.species)

10×424 Array{Bool,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

In [3]:
# list of nodes β ∈ graph that could possibly correpond with node α ∈ subgraph
function candidate_list(M::Array{Bool, 2}, α::Int)::Array{Int, 1}
    return [β for β = 1:size(M, 2) if M[α, β]]
end

cl = candidate_list(M₀, 1)

192-element Array{Int64,1}:
 137
 138
 139
 140
 141
 142
 143
 144
 145
 146
 147
 148
 149
   ⋮
 317
 318
 319
 320
 321
 322
 323
 324
 325
 326
 327
 328

In [4]:
# does node α have possible candidate matches in the graph?
function has_candidates(M::Array{Bool, 2}, α::Int)::Bool
    for β ∈ 1:size(M, 2) # loop over graph nodes
        if M[α, β]
            return true
        end
    end
    # if made it this far, no graph node could possible correspond to α ∈ subgraph =/
    return false 
end

has_candidates (generic function with 1 method)

In [5]:
function is_isomorphism(M::Array{Bool, 2})::Bool
    # (1) each row of M, corresponding to a node α ∈ subgraph, contains exactly one 1.
    #     i.e., every subgraph node has exactly one correspondence
    for α ∈ 1:size(M, 1)
        if sum(M[α, :]) != 1
            return false
        end
    end
    # (2) no column of M, corresponding to a node β ∈ graph, contains more than one 1.
    #     i.e., a graph node does not correspond to more than 1 subgraph node.
    for β ∈ 1:size(M, 2)
        if sum(M[:, β]) > 1
            return false
        end
    end
    # if made it this far, it is indeed an isomorphism.
    return true
end

is_isomorphism(M₀)

false

In [6]:
# idea here:
#   if any subgraph node α has no possible correspondence w/ a node β in the graph, no point in continuing
#   return true iff M has no empty candidate lists for subgraph nodes.
function possibly_contains_isomorphism(M::Array{Bool, 2})::Bool
    @debug "Validating M: $(M)"
    for α ∈ 1:size(M, 1) # loop over subgraph nodes
        if ! has_candidates(M, α)
            return false # subgraph node α cannot be assigned! =-O no point in continuing.
        end
    end
    return true # M may be the intersection of one or more solutions.
end

possibly_contains_isomorphism(M₀)

true

In [7]:
neighbors(xtal.bonds, 1)

0-element Array{Int64,1}

In [8]:
function prune!(M::Array{Bool, 2}, subgraph::SimpleGraph, graph::SimpleGraph)
    pruned = true # to enter while loop
    while pruned
        pruned = false
        for α ∈ size(M, 1) # loop thru subgraph nodes
            # get neighbors of node α
            neighbors_of_α = neighbors(subgraph, α)
            # loop thru candidate matches β ∈ graph for this subgraph node α
            for β ∈ candidate_list(M, α)
                neighbors_of_β = neighbors(graph, β)
                # now, suppose α ∈ subgraph and β ∈ graph correspond...
                for x ∈ neighbors_of_α
                    # if there is no neighbor of β that could correspond to x, neighbor of α, then, contradiction.
                    if length(intersect(candidate_list(M, x), neighbors_of_β)) == 0
                        M[α, β] = false
                        pruned = true
                    end
                end
            end
        end
    end
end

prune!(M₀, fragment.bonds, xtal.bonds)

In [9]:
size(M₀, 1)

10

In [13]:
# suppose node α ∈ subgraph and node β ∈ graph correspond.
#   modify M accordingly.
function assign_correspondence(M::Array{Bool, 2}, α::Int, β::Int)
    M[α, :] .= false # zero out row of subgraph node
    M[:, β] .= false # zero out column of graph node
    M[α, β] = true # assign correspondence at intersection
end

assign_correspondence(M₀, 1, 2)
M₀

10×424 Array{Bool,2}:
 0  1  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  1  1  1  1  1  1  1  1

In [16]:
# soln:
#    soln[α ∈ subgraph] = β ∈ graph where α corresponds to β
function depth_first_search(α::Int, subgraph::SimpleGraph, graph::SimpleGraph, M::Array{Bool, 2}, soln::Array{Int, 1}, β_mapped::Array{Bool, 1})
    if α > size(M, 1) || α < 1
        return nothing
    end
    
    # back-track
    if ! has_candidates(M, α)
        depth_first_search(α - 1, subgraph, graph, M, soln, β_mapped)
    end
    
    # loop thru un-assigned graph nodes β that could possibly correspond to subnode α
    for β ∈ candidate_list(M, α)
        if β_mapped[β] # if this graph node is already mapped...
            continue
        end
        M′ = copy(M)
        
        assign_correspondence(M′, α, β)
        soln[α] = β
        β_mapped[β] = true
        
        prune!(M′, subgraph, graph)
        
        # if we reached bottom of tree, iso-morphism is found!
        if α == size(M′, 1)
            # do we hv to check if it's a sol'n or is it guarenteed? why prune then?
            if is_isomorphism(M′)
                @info "found soln"
                push!(solns, soln)
            end
            # don't return b/c we need to look at other candidates
        end
        
        if possibly_contains_isomorphism(M′)
            # we've assigned α, go deeper
            depth_first_search(α + 1, subgraph, graph, M′, soln, β_mapped)
        end
        
        β_mapped[β] = false
        soln[α] = 0
    end
end

solns = []
soln = [0 for i = 1:nv(subgraph)]
β_mapped = [false for i = 1:nv(graph)]
depth_first_search(1, fragment.bonds, xtal.bonds, M₀, soln, β_mapped)

┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: found soln
└ @ Main In[16]:30
┌ Info: foun

InterruptException: InterruptException:

In [17]:
solns

123530-element Array{Any,1}:
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 ⋮
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139, 140, 257, 343, 341, 358, 353]
 [2, 137, 138, 139